## With Tensorflow Keras

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import spacy

2024-08-03 10:00:26.884432: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 10:00:26.884573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 10:00:27.003225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
start_char = 1
oov_char = 2
index_from = 3

In [3]:
(x_train, y_train), _ = tf.keras.datasets.imdb.load_data(start_char=start_char, oov_char=oov_char, index_from=index_from)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
x_train.shape

(25000,)

In [5]:
x_train.dtype

dtype('O')

In [6]:
y_train.shape

(25000,)

In [7]:
y_train[:5]

array([1, 0, 0, 1, 0])

In [8]:
df = pd.concat([pd.DataFrame({'X': x_train, 'Y': y_train})], ignore_index=True)
df.head()

,X,Y
0,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...",1
1,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...",0
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...",0
3,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1...",1
4,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...",0


In [9]:
word_index = tf.keras.datasets.imdb.get_word_index()

inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)

inverted_word_index[start_char] = ""
inverted_word_index[oov_char] = ""


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [10]:
print(len(word_index))
print(len(inverted_word_index))

88584
88586


In [11]:
encode = lambda text : [word_index[word] for word in text.split()]
decode = lambda inp : " ".join([inverted_word_index[idx] for idx in inp])

In [12]:
df["text"] = df["X"].apply(decode)

In [13]:
nlp = spacy.load("en_core_web_sm")
def remove_stop_words(text):
    doc = nlp(text)
    filtered = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered.append(token.lemma_)
    return " ".join(filtered)

In [ ]:
df["pre_text"] = df["text"].apply(remove_stop_words)

In [ ]:
nlp = spacy.load("en_core_web_lg")
df["vector"] = df["pre_text"].apply(lambda text: nlp(text).vector)

In [ ]:
df.head()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X = df["vector"].to_numpy()
X = np.array(X.tolist())
X.shape

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
Y = df["Y"].to_numpy().astype(np.uint8)

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
input_size = 300
num_classes = 2
batch_size = 16
dropout = 0.2

In [ ]:
model = Sequential([
    Dense(input_size, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(300, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(100, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(100, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(num_classes, activation="softmax")
])

In [ ]:
tf.config.list_physical_devices()

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)